<a href="https://colab.research.google.com/github/jackma-00/peft-of-a-llm/blob/main/lora_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth

In [2]:
# Also get the latest version of bitsandbytes
!pip install -U bitsandbytes

In [3]:
!pip install -U polygon-api-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.9 MB/s eta 0:00:00


In [4]:
from polygon import RESTClient
import json

# Your Polygon API Key
client = RESTClient(api_key="l3Ae5NltOnv2MqCVkfs62CSIKzWs_Z6H")

# List of tickers to track
tickers = ["SPY", "DIA", "QQQ", "IWM", "VXX"]

# Collect data in a structured format
data = []

for ticker in tickers:
    quote = client.get_previous_close_agg(ticker=ticker)

    # Access attributes directly from the first result
    formatted_quote = {
        "ticker": quote[0].ticker,
        "open": quote[0].open,
        "high": quote[0].high,
        "low": quote[0].low,
        "close": quote[0].close,
        "volume": quote[0].volume,
        "timestamp": quote[0].timestamp,
        "vwap": quote[0].vwap,
    }
    data.append(formatted_quote)

# Convert the data to a JSON string for LLM analysis
formatted_data = json.dumps(data, indent=4)
print(formatted_data)


[
    {
        "ticker": "SPY",
        "open": 599.52,
        "high": 600.86,
        "low": 595.2,
        "close": 597.53,
        "volume": 42365646.0,
        "timestamp": 1732568400000,
        "vwap": 597.7636
    },
    {
        "ticker": "DIA",
        "open": 446.61,
        "high": 448.4,
        "low": 445.4,
        "close": 447.56,
        "volume": 2892225.0,
        "timestamp": 1732568400000,
        "vwap": 447.2456
    },
    {
        "ticker": "QQQ",
        "open": 509.9,
        "high": 511.4525,
        "low": 504.2625,
        "close": 506.59,
        "volume": 26166808.0,
        "timestamp": 1732568400000,
        "vwap": 507.2417
    },
    {
        "ticker": "IWM",
        "open": 241.49,
        "high": 244.98,
        "low": 241.39,
        "close": 242.4,
        "volume": 33025021.0,
        "timestamp": 1732568400000,
        "vwap": 243.071
    },
    {
        "ticker": "VXX",
        "open": 44.39,
        "high": 46.2463,
        "low": 43.88,


In [7]:
print("Here are the aggregated values: {}".format(formatted_data))

Here are the aggregated values: [
    {
        "ticker": "SPY",
        "open": 599.52,
        "high": 600.86,
        "low": 595.2,
        "close": 597.53,
        "volume": 42365646.0,
        "timestamp": 1732568400000,
        "vwap": 597.7636
    },
    {
        "ticker": "DIA",
        "open": 446.61,
        "high": 448.4,
        "low": 445.4,
        "close": 447.56,
        "volume": 2892225.0,
        "timestamp": 1732568400000,
        "vwap": 447.2456
    },
    {
        "ticker": "QQQ",
        "open": 509.9,
        "high": 511.4525,
        "low": 504.2625,
        "close": 506.59,
        "volume": 26166808.0,
        "timestamp": 1732568400000,
        "vwap": 507.2417
    },
    {
        "ticker": "IWM",
        "open": 241.49,
        "high": 244.98,
        "low": 241.39,
        "close": 242.4,
        "volume": 33025021.0,
        "timestamp": 1732568400000,
        "vwap": 243.071
    },
    {
        "ticker": "VXX",
        "open": 44.39,
        "high":

In [12]:
from transformers import AutoModel, AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_name = "jackma-00/lora_model_1b" # Model name


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

system_prompt = "You are a financial consultant. Answer your client's questions using yesterday's closing aggregates for the following key tickers: {}".format(formatted_data)

messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": "How should I allocate additional 10k to my portfolio?"
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Based on the latest closing data provided for the specified tickers (SPY, DIA, QQQ, IWM, VXX) and considering today's market, it appears that there is no clear-cut way to allocate the additional $10,000. 

These are high-cap stocks that can potentially offer substantial long-term gains due to the growth and dividend payments, but the current market conditions may not align in your favor. The value of these stocks could potentially decrease further due to the volatility in the market.

At this point, a financial consultant may advise cautio